In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '1'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# probability model
from scipy.stats import multivariate_normal

In [ ]:
train_data = np.load("C:/Users/User/PycharmProjects/FL_AD/concat_UCF_V2.npy") #UCF
# train_data = np.load("/home/anas.al-lahham/AD_Unsupervised/concat_XD_I3D.npy") #XD
train_data.shape


In [ ]:
train_data1 = train_data[171:]
train_data1.shape

In [ ]:
np.save("concat_UCF_V2.npy", train_data1)

In [ ]:
train_data = train_data[:,0,0:1024]
# train_data = []

In [ ]:
nalist = np.load("C:/Users/User/PycharmProjects/FL_AD/nalist.npy") #UCF
nalist

In [ ]:
def estimate_gauss(X):
    m = X.shape[0]   # using only first dimension as we know it has only one feature - l2 norm
    
    mu = np.mean(X, axis=0)
    var = np.cov(X.T)
    
    return mu, var

In [ ]:
def estimate_gauss(X):
    m = X.shape[0]   # using only first dimension as we know it has only one feature - l2 norm
    
    mu = 1 / m * np.sum(X, axis=0)
    var = 1 / m * np.sum((X - mu) ** 2, axis=0)
    
    return mu, var

In [ ]:
new_repr = []
for i, (fromid, toid) in enumerate(nalist):
    new_repr.append(train_data[fromid:toid])

len(new_repr)

In [ ]:
new_repr[0].shape

In [ ]:
def get_matrix(data):
    # 1st: L2 norm [Euclidean Distance]
    # l2_norm = np.sum(np.square(data), axis=2)
    l2_norm = np.sum(np.square(data), axis=2)
    n_train_crop_l2_norm_mean = np.mean(l2_norm, axis= 1)
    # print(n_train_crop_l2_norm_mean.shape)
    
    # n_train_crop_l2_norm_mean_normal = Transform_normal(n_train_crop_l2_norm_mean)
    return n_train_crop_l2_norm_mean

In [ ]:
params = []
for i in range(len(new_repr)):
    # l2_norm = np.sum(np.square(new_repr[i]), axis=2)
    # mean_l2 =  np.mean(l2_norm, axis= 1)
    param = get_matrix(new_repr[i])
    mu, var = estimate_gauss(param)
    # print( mu, var )
    # l2mean = np.sum(np.square(new_repr[i]), a
    # xis=1).mean()
    # mmean = np.mean(new_repr[i], axis=0)
    params.append((mu, var, ))
len(params)

In [ ]:
params = np.load("C:/Users/User/PycharmProjects/FL_AD/params_UCF.npy")
params[1]

In [ ]:
import sklearn
print(sklearn.show_versions())

In [ ]:

from sklearn.mixture import GaussianMixture
# import time


gmm = GaussianMixture(n_components=2, max_iter=50, random_state=0, covariance_type='spherical')
# gmm_scores = gmm.score_samples(params)
labels = gmm.fit_predict(params)

# y_gmm = gmm.fit_predict(params)
# print(y_gmm.sum(), y_gmm.sum() / len(y_gmm))

# score = y_gmm 
# score = gmm.score_samples(params) 
# pct_threshold = np.percentile(score, 3)
# print(f'The threshold of the score is {pct_threshold:.2f}') 
# res = np.array([1 if x < pct_threshold else 0 for x in score]) 
# print(res.sum())
np.where(labels== 1)[0].shape, np.where(labels == 0)[0].shape

In [ ]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN

In [ ]:
KM = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(params[:200])
KM.labels_

In [ ]:
db = DBSCAN(eps=150, min_samples=5 )
db.fit_predict(params[:])

In [ ]:
db.labels_
np.where(db.labels_ == -1)[0].shape, np.where(db.labels_ == 0)[0].shape

In [ ]:
len(db.labels_)

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=2, max_iter=500,covariance_type= 'spherical', tol=1e-9).fit(params)
gmm_scores = gmm.score_samples(params)
labels = gmm.predict(params)

In [ ]:
abnormal_portion = np.where(res == 1)[0]
normal_portion = np.where(res == 0)[0]
normal_portion.shape, abnormal_portion.shape

In [ ]:
n_params = np.array(params)[normal_portion]
a_params = np.array(params)[abnormal_portion]
n_params.shape, a_params.shape

In [ ]:
from sklearn.mixture import GaussianMixture
gmm1 = GaussianMixture(n_components=2, max_iter=100,covariance_type= 'spherical', tol=1e-9)
# gmm_scores = gmm1.score_samples(n_params)
labels1 = gmm1.fit_predict(params)

In [ ]:
np.where(labels1 ==0)[0].shape, np.where(labels1 ==1)[0].shape

In [ ]:
video_labels_1 = labels
video_labels_2 = labels1
video_labels_1.shape, video_labels_2.shape

In [ ]:
video_labels_2


In [ ]:
a = np.full(video_labels_1.shape[0] - video_labels_2.shape[0] ,1)
video_labels_2 = np.concatenate((video_labels_2, a))
video_labels_1.shape, video_labels_2.shape

In [ ]:
abnormal_portion = np.where(video_labels_2 == 0)[0]
normal_portion = np.where(video_labels_2 == 1)[0]
abnormal_portion.shape, normal_portion.shape

In [ ]:
abnormal_portion = np.where(video_labels_2 == 1)[0]
normal_portion = np.where(video_labels_2 == 0)[0]

In [ ]:
abag = list(zip(list(np.array(params)[abnormal_portion]), abnormal_portion))
nbag = list(zip(list(np.array(params)[normal_portion]), normal_portion))
len(abag), len(nbag)


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [ ]:
nu = 1.0
step = 1
import time
start = time.time()
while len(abag) / len(nbag) < nu:
    
    temp_bag = nbag
    y_gmm = gmm.fit_predict([list(x[0]) for x in np.array(temp_bag)])
    score = y_gmm 
    score = gmm.score_samples([list(x[0]) for x in np.array(temp_bag)]) 
    pct_threshold = np.percentile(score, 3) 
    res = np.array([1 if x < pct_threshold else 0 for x in score]) 
    print(f'The threshold of the score in step {step} is {pct_threshold:.2f}, abnormal part: {res.sum()}') 
    
    abnormal_portion = np.where(res == 1)[0]
    normal_portion = np.where(res == 0)[0]
    
    abag += [(x[0], x[1]) for x in np.array(temp_bag)[abnormal_portion]]
    nbag = [(x[0], x[1]) for x in np.array(temp_bag)[normal_portion]]

    step += 1

print(time.time() - start)

In [ ]:

abnormal_portion = np.where(res == 1)[0]
normal_portion = np.where(res == 0)[0]
abag = list(zip(list(np.array(params)[abnormal_portion]), abnormal_portion))
nbag = list(zip(list(np.array(params)[normal_portion]), normal_portion))
len(abag), len(nbag)



In [ ]:
step = 1
while len(abag) / len(nbag) < nu:
    
    temp_bag = nbag
    y_gmm = gmm.fit_predict([list(x[0]) for x in np.array(temp_bag)])
    score = y_gmm 
    score = gmm.score_samples([list(x[0]) for x in np.array(temp_bag)]) 
    pct_threshold = np.percentile(score, 1.) 
    res = np.array([1 if x < pct_threshold else 0 for x in score]) 
    print(f'The threshold of the score in step {step} is {pct_threshold:.2f}, abnormal part: {res.sum()}') 
    
    abnormal_portion = np.where(res == 1)[0]
    normal_portion = np.where(res == 0)[0]
    
    abag += [(x[0], x[1]) for x in np.array(temp_bag)[abnormal_portion]]
    nbag = [(x[0], x[1]) for x in np.array(temp_bag)[normal_portion]]

    step += 1

In [ ]:
len(abag), len(nbag)

In [ ]:
print(np.where(np.array([x[1] for x in abag]) < 810)[0].shape, len([x[1] for x in abag]))
print('correctness acc: ', np.where(np.array([x[1] for x in abag]) < 810)[0].shape[0] / len([x[1] for x in abag]))

In [ ]:
print(np.where(np.array([x[1] for x in nbag]) > 810)[0].shape, len([x[1] for x in nbag]))
print('correctness acc: ', np.where(np.array([x[1] for x in nbag]) > 810)[0].shape[0] / len([x[1] for x in nbag]))

In [ ]:
np.where(res ==0)[0].shape, np.where(res ==1)[0].shape

In [ ]:
video_labels_1 = labels
video_labels_2 = labels1
video_labels_1.shape, video_labels_2.shape

In [ ]:
a = np.full(video_labels_1.shape[0] - video_labels_2.shape[0] ,1)
video_labels_2 = np.concatenate((video_labels_2, a))
video_labels_1.shape, video_labels_2.shape

In [ ]:
temp = [k[1] for k in sorted([(x[1], 1.0) for x in abag] + [(x[1], 0.0) for x in nbag], key=lambda z: z[0])]
sum(temp), len(temp)

In [ ]:
# normal set creation
normal_set = {}

for i in range(len(new_repr)):
    if video_labels_2[i] == 0.0:
        normal_set[i] = new_repr[i]

In [ ]:
# abnormal set creation
abnormal_set = {}
for i in range(len(new_repr)):
    if video_labels_2[i] == 1.0:
        abnormal_set[i] = new_repr[i]

In [ ]:
l2_norms_N = np.empty(0,)
for (idel, sample) in normal_set.items():
    
    # print(sample.shape)
    

    l2_norms_N = np.append(l2_norms_N,get_matrix(sample))

In [ ]:
len(normal_set), len(abnormal_set)

In [ ]:
mu_GMM, var_GMM = estimate_gauss(np.array(l2_norms_N))

In [ ]:
# probability model
from scipy.stats import multivariate_normal
p = multivariate_normal(mu_GMM, var_GMM)

In [ ]:
ground_truth = {} 
length = 0.2 
for (idel, sample) in abnormal_set.items(): 

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    probs = p.pdf(sample_matrix)
    temp_list = []
    temp_list += [0.0] * len(probs)
    
    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth[idel] = temp_list

In [ ]:
final_gt = []
abnormal_gt = []
for i in range(len(new_repr)):
    if i in normal_set.keys():
        final_gt += [0.0] * new_repr[i].shape[0]
    else:
        final_gt += ground_truth[i]
        abnormal_gt+= ground_truth[i]

In [ ]:
len(final_gt)

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_V2_w_20.npy', final_gt) #UCF
# np.save('Unsup_labels/'+'XD_I3D_unsup_labels_10_V2_GMM.npy', final_gt) #XD

## Random Video-level labels experiment 

In [ ]:
# generate 1609 random weak labels either 0 or 1
rand_temp = np.random.randint(2, size=1609)
rand_temp

In [ ]:
# normal set creation
normal_set_RS1FPL = {}

for i in range(len(new_repr)):
    if rand_temp[i] == 0:
        normal_set_RS1FPL[i] = new_repr[i]

In [ ]:
# abnormal set creation
abnormal_set_RS1FPL = {}
for i in range(len(new_repr)):
    if rand_temp[i] == 1:
        abnormal_set_RS1FPL[i] = new_repr[i]

In [ ]:
l2_norms_N_RS1FPL = np.empty(0,)
for (idel, sample) in normal_set_RS1FPL.items():
    
    # print(sample.shape)

    l2_norms_N_RS1FPL = np.append(l2_norms_N_RS1FPL,get_matrix(sample))


mu_GMM_RS1FPL, var_GMM_RS1FPL = estimate_gauss(np.array(l2_norms_N_RS1FPL))
mu_GMM_RS1FPL, var_GMM_RS1FPL

In [ ]:
p_RS1FPL = multivariate_normal(mu_GMM_RS1FPL, var_GMM_RS1FPL)

In [ ]:
ground_truth_RS1FPL = {} 
length = 0.2 

for (idel, sample) in abnormal_set.items(): 

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    probs = p_RS1FPL.pdf(sample_matrix)
    temp_list = []
    temp_list += [0.0] * len(probs)
    
    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth_RS1FPL[idel] = temp_list

len(ground_truth_RS1FPL)

In [ ]:
final_gt_RS1FPL = []
abnormal_gt_RS1FPL = []
for i in range(len(new_repr)):
    if i in normal_set_RS1FPL.keys():
        final_gt_RS1FPL += [0.0] * new_repr[i].shape[0]
    else:
        final_gt_RS1FPL += ground_truth_RS1FPL[i]
        abnormal_gt_RS1FPL+= ground_truth_RS1FPL[i]

len(final_gt_RS1FPL)

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_RS1FPL.npy', final_gt_RS1FPL) #UCF


## Random Video-level labels experiment 

In [ ]:
con_test = np.load("/home/anas.al-lahham/AnomalyDetection/RTFM/10_crop_features/Concat_test_10.npy")

In [ ]:
con_test.shape

In [ ]:
n_train_crop_features = get_matrix(train_data[:580699])
mu, var = estimate_gauss(n_train_crop_features)

In [ ]:
p = multivariate_normal(mu, var)

In [ ]:
videos_list = '/home/anas.al-lahham/AD_Unsupervised/list/ucf-i3d-test.list'
videos_list = list(open(videos_list))

In [ ]:
cnt = 0
tlist = []
for filename in videos_list:
    filename = filename.strip('\n')
    sample = np.load(filename)

    length = len(sample)
    tlist.append((cnt, cnt + length))
    cnt += length

In [ ]:
new_repr_test = []
for i, (fromid, toid) in enumerate(tlist):
    new_repr.append(con_test[fromid:toid])

In [ ]:
ground_truth = [] 
length = 0.2 
for filename in videos_list: 
    filename = filename.strip('\n')
    sample = np.load(filename)

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    # get p values
    probs = p.pdf(sample_matrix)



    temp_list = []
    temp_list += [0.0] * len(probs)
    

    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth += temp_list

In [ ]:
len(ground_truth)

In [ ]:
from sklearn.metrics import auc, roc_curve, precision_recall_curve


In [ ]:
np.array(ground_truth)

In [ ]:
gt = np.load('list/gt-ucf-RTFM.npy')
# print(gt.shape)
pred = list(np.array(ground_truth))
pred = np.repeat(np.array(pred), 16)
# gt = gt[:len(pred)] 

fpr, tpr, threshold = roc_curve(list(gt), pred)


rec_auc = auc(fpr, tpr)
rec_auc

In [ ]:
ground_truth = np.concatenate((np.zeros(580699),np.ones(779951-580699)))
ground_truth.shape

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_GTVSPL.npy', ground_truth)

### Video-level pseudo-labels are directly attributed to the segments without refinement


In [ ]:
pseudo_labels = np.load('Unsup_labels/UCF_unsup_labels_original_V2.npy')
pseudo_labels.shape

In [ ]:
all_list = list(open('list/ucf-i3d.list'))

In [ ]:
cnt = 0
nlist = []
for video in all_list:
    location = video.strip('\n')
    sample = np.load(location)

    length = len(sample)
    nlist.append((cnt, cnt + length))
    

In [ ]:
cnt = 0
GTVSPL_ground_truth = []
for video in all_list:
    location = video.strip('\n')
    sample = np.load(location)

    length = len(sample)
    if np.sum(pseudo_labels[cnt:cnt+length]) == 0:
        GTVSPL_ground_truth+= [0.0] * length

    else:
        GTVSPL_ground_truth+= [1.0] * length
    cnt += length
len(GTVSPL_ground_truth)

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_GTVSPL.npy', GTVSPL_ground_truth)

In [ ]:
np.sum(GTVSPL_ground_truth)

### (1 – p-value) is used as anomaly score


In [ ]:
pseudo_labels = np.load('Unsup_labels/UCF_unsup_labels_original_V2.npy')
pseudo_labels[:].sum()

In [ ]:
cnt = 0
normal_set_C2PLnoAD = {}
abnormal_set_C2PLnoAD = {}
for i,video in enumerate(all_list):
   location = video.strip('\n')
   sample = np.load(location)

   length = len(sample)
   if np.sum(pseudo_labels[cnt:cnt+length]) == 0:
      normal_set_C2PLnoAD[i] = new_repr[i]
   else:
      abnormal_set_C2PLnoAD[i] = new_repr[i]
      


   cnt += length
    # break

In [ ]:
len(normal_set_C2PLnoAD), len(abnormal_set_C2PLnoAD)

In [ ]:
1344+265

In [ ]:
l2_norms_N_C2PLnoAD = np.empty(0,)
for (idel, sample) in normal_set_C2PLnoAD.items():
    
    # print(sample.shape)

    l2_norms_N_C2PLnoAD = np.append(l2_norms_N_C2PLnoAD,get_matrix(sample))


mu_GMM_C2PLnoAD , var_GMM_C2PLnoAD  = estimate_gauss(np.array(l2_norms_N_C2PLnoAD))
mu_GMM_C2PLnoAD , var_GMM_C2PLnoAD 

In [ ]:
p_C2PLnoAD  = multivariate_normal(mu_GMM_C2PLnoAD , var_GMM_C2PLnoAD )

In [ ]:
videos_list = '/home/anas.al-lahham/AD_Unsupervised/list/ucf-i3d-test.list'
videos_list = list(open(videos_list))

In [ ]:
ground_truth = [] 
p_value = []
length = 0.2 
for filename in videos_list: 
    filename = filename.strip('\n')
    sample = np.load(filename)

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    # get p values
    probs = p_C2PLnoAD.pdf(sample_matrix)
    
    p_value = np.append(p_value, probs)

    temp_list = []
    temp_list += [0.0] * len(probs)
    

    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth += temp_list

In [ ]:
p_value

In [ ]:
for k in range(10, 55, 5):


    ground_truth_C2PLnoAD = {} 
    length = k/100

    for (idel, sample) in abnormal_set_C2PLnoAD.items(): 

        # feature extraction 
        # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
        sample_matrix = get_matrix(sample)
        
        # get p values
        probs = p_C2PLnoAD.pdf(sample_matrix)
        temp_list = []
        temp_list += [0.0] * len(probs)
        
        window_size = int(len(probs) * length)  # fixed
        temp = []
        for idx in range(0, len(probs) - window_size + 1):
            arr = 0
            for i in range(idx, idx + window_size - 1):
                arr += abs(probs[i+1] - probs[i])
            temp.append(arr)
        # print(temp)
        for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
            temp_list[i] = 1.0

        ground_truth_C2PLnoAD[idel] = temp_list

    

    final_gt_C2PLnoAD = []
    abnormal_gt_C2PLnoAD = []
    for i in range(len(new_repr)):
        if i in normal_set_C2PLnoAD.keys():
            final_gt_C2PLnoAD += [0.0] * new_repr[i].shape[0]
        else:
            final_gt_C2PLnoAD += ground_truth_C2PLnoAD[i]
            abnormal_gt_C2PLnoAD+= ground_truth_C2PLnoAD[i]

    


    np.save('unsupervised_PL/'+f'UCF_labels_best_w_{k}.npy', final_gt_C2PLnoAD) #UCF

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_best_w_15.npy', final_gt_C2PLnoAD) #UCF

In [ ]:
gt = np.load('list/gt-ucf-RTFM.npy')
# print(gt.shape)
# pred = list(np.array(ground_truth))
pred = np.repeat(np.array(p_value), 16)
# gt = gt[:len(pred)] 

fpr, tpr, threshold = roc_curve(list(gt), pred)


rec_auc = auc(fpr, tpr)
rec_auc

In [ ]:
ground_truth = [] 
p_value = []
length = 0.2 
for filename in videos_list: 
    filename = filename.strip('\n')
    sample = np.load(filename)

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    # get p values
    probs = p.pdf(sample_matrix)
    
    p_value = np.append(p_value, probs)

    temp_list = []
    temp_list += [0.0] * len(probs)
    

    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth += temp_list

In [ ]:
ground_truth = [] 

length = 0.15 
for filename in all_list: 
    filename = filename.strip('\n')
    sample = np.load(filename)

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    # get p values
    probs = p.pdf(sample_matrix)
    


    temp_list = []
    temp_list += [0.0] * len(probs)
    

    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth += temp_list

In [ ]:
len(ground_truth)

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_OCC_w_15.npy', ground_truth) #UCF